In [9]:
import pandas as pd
import numpy as np
import random
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import classification_report
from sklearn.multioutput import MultiOutputClassifier
import joblib

# ===============================
# 1. Pembuatan Dataset
# ===============================

# Atur parameter
jumlah_user = 300           # Jumlah pengguna
jumlah_kelas = 5            # Jumlah kelas
jumlah_materi_per_kelas = 8  # Jumlah materi/topik dalam setiap kelas
ambang_batas_rekomendasi = 60  # Skor minimum untuk tidak direkomendasikan

# Buat user_id dan class_id
user_ids = [f"U{str(i).zfill(3)}" for i in range(1, jumlah_user + 1)]
class_ids = [f"C{str(i).zfill(2)}" for i in range(1, jumlah_kelas + 1)]

# Nama materi (topik) dalam setiap kelas
topics = [f"Topic_{i}" for i in range(1, jumlah_materi_per_kelas + 1)]

data = []

for user in user_ids:
    # Setiap user ikut 1 atau 2 kelas secara acak
    ikut_kelas = random.sample(class_ids, random.choice([1, 2]))

    for kelas in ikut_kelas:
        # Generate skor kuis untuk setiap materi
        quiz_scores = {topic: random.randint(50, 100) for topic in topics}

        # Tambahkan fitur tambahan: waktu pengerjaan kuis (dalam menit)
        quiz_times = {f"{topic}_time": random.uniform(5, 30) for topic in topics}  # Waktu antara 5-30 menit

        # Tambahkan fitur tambahan: jumlah percobaan
        attempts = {f"{topic}_attempts": random.randint(1, 3) for topic in topics}  # 1-3 percobaan

        # Tentukan materi yang direkomendasikan
        recommended_topics = [topic for topic, score in quiz_scores.items() if score < ambang_batas_rekomendasi]

        # Gabungkan semua data
        row = {
            "user_id": user,
            "class_id": kelas,
            **quiz_scores,
            **quiz_times,
            **attempts,
            "recommended_topics": ", ".join(recommended_topics) if recommended_topics else "None"
        }
        data.append(row)

# Buat DataFrame
df = pd.DataFrame(data)

# Simpan ke CSV
df.to_csv("recommendation_dataset_enhanced.csv", index=False)
print("✅ Dataset berhasil dibuat dan disimpan sebagai 'recommendation_dataset_enhanced.csv'")

# Tampilkan 5 baris pertama
print(df.head())

# ===============================
# 2. Pelatihan Model
# ===============================

# Load dataset dari file CSV
df = pd.read_csv("recommendation_dataset_enhanced.csv")

# Hapus semua kolom yang berisi '_time'
df = df.drop(columns=[col for col in df.columns if '_time' in col], errors='ignore')

# Pisahkan fitur dan label target
# Fitur adalah skor kuis dan jumlah percobaan
jumlah_materi_per_kelas = 8  # Jumlah materi/topik dalam setiap kelas

feature_columns = (
    [f"Topic_{i}" for i in range(1, jumlah_materi_per_kelas + 1)] +
    [f"Topic_{i}_attempts" for i in range(1, jumlah_materi_per_kelas + 1)]
)

X = df[feature_columns]

# Label target adalah kolom 'recommended_topics'
# Ubah string yang dipisahkan koma menjadi list
# Handle NaN values dengan menggantinya dengan "None"
df['recommended_topics'] = df['recommended_topics'].fillna("None")  # Ganti NaN dengan "None"
df['recommended_topics'] = df['recommended_topics'].apply(lambda x: x.split(", ") if x != "None" else [])

# Gunakan MultiLabelBinarizer untuk mengonversi label ke format biner
mlb = MultiLabelBinarizer()
y = mlb.fit_transform(df['recommended_topics'])

# Split data menjadi data latih dan data uji
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Latih model menggunakan GradientBoostingClassifier
model = MultiOutputClassifier(GradientBoostingClassifier(random_state=42))
model.fit(X_train, y_train)

# Prediksi pada data uji
y_pred = model.predict(X_test)

# Evaluasi model
print("Classification Report:")
print(classification_report(y_test, y_pred, target_names=mlb.classes_))

# Simpan model dan MultiLabelBinarizer
joblib.dump(model, "recommendation_model_no_time.pkl")
joblib.dump(mlb, "multi_label_binarizer_no_time.pkl")

✅ Dataset berhasil dibuat dan disimpan sebagai 'recommendation_dataset_enhanced.csv'
  user_id class_id  Topic_1  Topic_2  Topic_3  Topic_4  Topic_5  Topic_6  \
0    U001      C05       95       98       79       82       56       95   
1    U001      C02       93       62       59       75       52       73   
2    U002      C02       84       56       83       65       64       74   
3    U003      C05       75       93       76       87       94       91   
4    U003      C01       65       92       92       69       59       96   

   Topic_7  Topic_8  ...  Topic_8_time  Topic_1_attempts  Topic_2_attempts  \
0       93       57  ...      7.519131                 2                 3   
1       77       68  ...     12.031200                 2                 1   
2       83       57  ...     17.148495                 1                 2   
3       84       58  ...     28.053903                 3                 2   
4       74       94  ...      8.660456                 1            

/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in samples with no true labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in samples with no true nor predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


['multi_label_binarizer_no_time.pkl']